<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ تثبيت icrawler
!pip install -q icrawler

# ✅ استيراد المكتبات
import os
from icrawler.builtin import GoogleImageCrawler
from tqdm import tqdm

# ✅ إنشاء مجلد لحفظ الصور
save_dir = "shark_fins"
os.makedirs(save_dir, exist_ok=True)

# ✅ قائمة الأنواع (الاسم الإنجليزي + الاسم العلمي)
shark_species = [
    ("Scalloped hammerhead", "Sphyrna lewini"),
    ("Great hammerhead", "Sphyrna mokarran"),
    ("Oceanic whitetip shark", "Carcharhinus longimanus"),
    ("Tiger shark", "Galeocerdo cuvier"),
    ("Silky shark", "Carcharhinus falciformis"),
    ("Blacktip shark", "Carcharhinus limbatus"),
    ("Dusky shark", "Carcharhinus obscurus"),
    ("Grey reef shark", "Carcharhinus amblyrhynchos"),
    ("Thresher shark", "Alopias vulpinus"),
    ("Shortfin mako", "Isurus oxyrinchus")
]

# ✅ المواقع الموثوقة
trusted_sources = [
    "site:wikipedia.org",
    "site:inaturalist.org",
    "site:marinespecies.org",
    "site:noaa.gov",
    "site:arkive.org",
    "site:shark-references.com"
]

# ✅ دالة لتحميل الصور من استعلام معين باستخدام icrawler
def download_images(query, species_folder, limit=500):
    os.makedirs(species_folder, exist_ok=True)
    try:
        google_crawler = GoogleImageCrawler(storage={'root_dir': species_folder})
        google_crawler.crawl(keyword=query, max_num=limit, file_idx_offset='auto')
    except Exception as e:
        print(f"⚠️ فشل تحميل صور للاستعلام: {query}\nالخطأ: {e}")

# ✅ تنفيذ التحميل لكل نوع قرش
for common, scientific in shark_species:
    folder_name = os.path.join(save_dir, common.replace(" ", "_"))
    print(f"⬇️ تحميل الصور لـ: {common} ({scientific})...")
    for site in trusted_sources:
        # تحميل الصور بالاسم الشائع
        query_common = f"{common} dorsal fin {site}"
        download_images(query_common, folder_name)
        # تحميل الصور بالاسم العلمي
        query_scientific = f"{scientific} dorsal fin {site}"
        download_images(query_scientific, folder_name)

print("✅ تم الانتهاء من تحميل الصور.")

In [ ]:
import os
import shutil
import random

source_dir = "shark_fins"
target_dir = "shark_dataset"
split_ratio = 0.8  # 80% تدريب، 20% تحقق

for species in os.listdir(source_dir):
    src_path = os.path.join(source_dir, species)
    imgs = os.listdir(src_path)
    random.shuffle(imgs)

    split_index = int(len(imgs) * split_ratio)
    train_imgs = imgs[:split_index]
    val_imgs = imgs[split_index:]

    for phase, image_list in [('train', train_imgs), ('val', val_imgs)]:
        dest_folder = os.path.join(target_dir, phase, species)
        os.makedirs(dest_folder, exist_ok=True)
        for img_file in image_list:
            shutil.copy(os.path.join(src_path, img_file), os.path.join(dest_folder, img_file))

In [ ]:
def get_folder_size(path):
    total_size = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # بالميجابايت

train_size = get_folder_size("shark_dataset/train")
val_size = get_folder_size("shark_dataset/val")

print(f"📦 حجم train: {train_size:.2f} MB")
print(f"📦 حجم val  : {val_size:.2f} MB")

📦 حجم train: 278.73 MB
📦 حجم val  : 81.55 MB


In [ ]:
import os

def count_images_in_folders(base_path):
    print(f"\n📂 تحليل المجلد: {base_path}\n" + "-"*40)
    total_images = 0
    for species in sorted(os.listdir(base_path)):
        folder_path = os.path.join(base_path, species)
        if os.path.isdir(folder_path):
            num_images = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            total_images += num_images
            print(f"{species:<30} : {num_images} صورة")
    print("-"*40)
    print(f"📊 الإجمالي: {total_images} صورة\n")

count_images_in_folders("shark_dataset/train")
count_images_in_folders("shark_dataset/val")


📂 تحليل المجلد: shark_dataset/train
----------------------------------------
Blacktip_shark                 : 0 صورة
Dusky_shark                    : 0 صورة
Great_hammerhead               : 251 صورة
Grey_reef_shark                : 0 صورة
Oceanic_whitetip_shark         : 245 صورة
Scalloped_hammerhead           : 254 صورة
Shortfin_mako                  : 0 صورة
Silky_shark                    : 0 صورة
Thresher_shark                 : 0 صورة
Tiger_shark                    : 210 صورة
----------------------------------------
📊 الإجمالي: 960 صورة


📂 تحليل المجلد: shark_dataset/val
----------------------------------------
Blacktip_shark                 : 0 صورة
Dusky_shark                    : 0 صورة
Great_hammerhead               : 63 صورة
Grey_reef_shark                : 0 صورة
Oceanic_whitetip_shark         : 62 صورة
Scalloped_hammerhead           : 64 صورة
Shortfin_mako                  : 0 صورة
Silky_shark                    : 0 صورة
Thresher_shark                 : 0 صورة
Tiger_shark 

In [ ]:
import shutil

def remove_empty_folders(base_path):
    for species in os.listdir(base_path):
        folder_path = os.path.join(base_path, species)
        if os.path.isdir(folder_path) and len(os.listdir(folder_path)) == 0:
            shutil.rmtree(folder_path)
            print(f"🗑️ حذف المجلد الفارغ: {folder_path}")

remove_empty_folders("shark_dataset/train")
remove_empty_folders("shark_dataset/val")

🗑️ حذف المجلد الفارغ: shark_dataset/train/Silky_shark
🗑️ حذف المجلد الفارغ: shark_dataset/train/Shortfin_mako
🗑️ حذف المجلد الفارغ: shark_dataset/train/Blacktip_shark
🗑️ حذف المجلد الفارغ: shark_dataset/train/Grey_reef_shark
🗑️ حذف المجلد الفارغ: shark_dataset/train/Thresher_shark
🗑️ حذف المجلد الفارغ: shark_dataset/train/Dusky_shark
🗑️ حذف المجلد الفارغ: shark_dataset/val/Silky_shark
🗑️ حذف المجلد الفارغ: shark_dataset/val/Shortfin_mako
🗑️ حذف المجلد الفارغ: shark_dataset/val/Blacktip_shark
🗑️ حذف المجلد الفارغ: shark_dataset/val/Grey_reef_shark
🗑️ حذف المجلد الفارغ: shark_dataset/val/Thresher_shark
🗑️ حذف المجلد الفارغ: shark_dataset/val/Dusky_shark


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch import nn, optim
from torch.utils.data import DataLoader

# ✅ التحويلات
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ✅ تحميل البيانات
train_dir = "shark_dataset/train"
val_dir = "shark_dataset/val"

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ✅ تحميل ResNet18
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# ✅ تعديل الطبقة النهائية
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# ✅ التدريب
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# ✅ حلقة التدريب
for epoch in range(5):  # عدد قليل للتجربة الأولية
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"✅ Epoch {epoch+1}, Loss: {total_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 44.4MB/s]
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Epoch 1, Loss: 42.9185
✅ Epoch 2, Loss: 40.3105
✅ Epoch 3, Loss: 39.2322
✅ Epoch 4, Loss: 37.9232
✅ Epoch 5, Loss: 37.1190


In [ ]:
# ✅ التقييم
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"🎯 دقة النموذج على مجموعة التحقق: {accuracy:.2f}%")

🎯 دقة النموذج على مجموعة التحقق: 33.06%


In [ ]:
import shutil

shutil.make_archive("shark_fins_raw", 'zip', "shark_fins")

'/content/shark_fins_raw.zip'